## 1.哈希表


### 1.转换思路之 数组中选两数求和为目标值---a+b=target
- 创建一个字典，将nums中的元素作为key，索引作为value
- 遍历nums，判断target-nums[i]是否在字典中，是则返回结果,否，则将nums[i]作为key，索引作为value加入字典!
- 时间复杂度：O(n)
- 空间复杂度：O(n)

In [ ]:
n,target=map(int,input().split())
nums=list(map(int,input().split()))
dic={}
for i in range(n):
    complement=target-nums[i]
    if complement in dic:
        print(dic[complement],i)
        exit()
    dic[nums[i]]=i

### 2.怎么选取健和值--字母异位词分组
思路:
1. 创建一个字典，key为排序后的字符串，value为列表，列表中存放的是原字符串---这样异位词组的健就是一样的!
=================================================================================
2. 遍历字符串列表，将字符串排序，并作为key，将原字符串作为value添加到字典中
3. 遍历字典，将value列表中的字符串添加到一个新的列表中，并返回新的列表

学:
1.注意!:sorted(s) 返回的是列表（list），而列表是不可哈希的,所以不能作为字典的key--要用key="".join(sorted(s))
2.用collections中的defaultdict来创建字典--默认字典，字典的key不存在时，会创建一个空的列表，并返回该列表
3."".join(Listt)来将列表Listt中的元素连接成一个字符串

In [ ]:
from collections import defaultdict
class solution:
    def groupanagrams(self,strs):
        mp=defaultdict(list)
        for s in strs:
            key="".join(sorted(s))
            mp[key].append(s)
        return list(mp.values())
if __name__=="__main__":
    n=int(input())
    strs=input().split()
    so=solution()
    res=so.groupanagrams(strs)
    for group in res:
        #print(" ".join(group))
        print(*group)

### 3.巨帅的贪心哈希解法--最长连续序列

学:

1.**利用set(nums)方便查找nums[i]-1/nums[i]+1是否在nums中---set()查找效率高O(1)!!!!!**

2.nums[i]-1不在set(nums)中说明其就是一个序列的起点

3.nums[i]+1在set(nums)中,说明该序列长度可以增加--le+=1,一直到找不到为止就是nums[i]序列的最终连续长度

4.maxlen = max(maxlen, le)找到最长长度

In [ ]:
from collections import deque
class solution:
    def maxsequence(self,nums):
        maxle=0
        setnums=set(nums)
        for i in range(len(nums)):
            if (nums[i]-1) not in setnums:
                le=1
                while (nums[i]+le) in setnums:
                    le+=1
                maxle=max(maxle,le)
            else:
                continue
        return maxle

so=solution()
n=int(input())
nums=list(map(int,input().split()))
print(so.maxsequence(nums))


## 2.双指针

### 1.同起点/紧挨着的双指针--移动零

两种思路:
1.挨个判断0移动到后面---需要考虑更多边界条件
2.移动非0到前面---更简洁,但会自己移动自己

In [ ]:
# 移动0 -后移
class solution:
    def movezero(self,nums):
        left=0
        right=1
        while right<len(nums):
            if nums[left]==0 and nums[right]!=0:
                nums[left],nums[right]=nums[right],nums[left]
                right+=1
                left+=1
            elif nums[left]==0:
                if nums[right]==0:
                    right+=1
            else:
                right+=1
                left+=1
so=solution()
n=int(input())
nums=list(map(int,input().split()))
so.movezero(nums)
print(*nums)



In [ ]:
# 移动非0--前移
class Solution:
    def moveZeroes(self, nums):
        n = len(nums)
        i = 0  # 记录非零元素应放置的位置
        
        # 遍历数组，将非零元素移到前面
        for j in range(n):
            if nums[j] != 0:
                nums[i], nums[j] = nums[j], nums[i]  # 交换非零元素到前面
                i += 1  # 移动非零元素位置索引

# 读入数据并调用函数
n = int(input().strip())
nums = list(map(int, input().split()))
Solution().moveZeroes(nums)
print(" ".join(map(str, nums)))


### 2.两头指针移动之优化(贪心)多移动其中一头--盛水最多容器

学:
1.我在想优化双指针，移动指针的时候跳过比当前短板更短的时候,循环没写好容易写成死循环,要记住这样写!:
    cur_left=nums[left] #要在循环外定义cur_left,方便比较
    while left < right and nums[left]<=cur_left:
                    left+=1 #只在循环里面叠加
        不要忘记判断left<right且将nums[left]<=cur_left:放入同级别的while中

In [ ]:
class solution:
    def maxwater(self,nums):
        left=0
        right=len(nums)-1
        res=0        
        while left<right:
            v=min(nums[left],nums[right])*(right-left)
            res=max(res,v)
            if nums[left]<nums[right]:
                cur_left=nums[left]
                while left < right and nums[left]<=cur_left:
                    left+=1
                    
            else:
                cur_right=nums[right]
                while left < right and nums[right]<=cur_right:
                    right-=1
                        
        return res
so=solution()
n=int(input())
nums=list(map(int,input().split()))
print(so.maxwater(nums))
                